In [1]:
import pandas as pd
import numpy as np
import sklearn 

In [8]:
#read in
demographics = pd.read_csv("data/DEMOGRAPHICS.csv")
summary = pd.read_csv("data/SUMMARYMEASURESOFHEALTH.csv")
leading = pd.read_csv("data/LEADINGCAUSESOFDEATH.csv")
measure = pd.read_csv("data/MEASURESOFBIRTHANDDEATH.csv")
vunerable  = pd.read_csv("data/VUNERABLEPOPSANDENVHEALTH.csv")
risk = pd.read_csv("data/RISKFACTORSANDACCESSTOCARE.csv")

In [ ]:
#merge
full = pd.concat([demographics, summary,leading, measure, vunerable,risk], axis=1, join="inner")

In [18]:
full.describe()

,State_FIPS_Code,County_FIPS_Code,Strata_ID_Number,Number_Counties,Population_Size,Min_Population_Size,Max_Population_Size,Population_Density,Min_Population_Density,Max_Population_Density,...,Diabetes,CI_Min_Diabetes,CI_Max_Diabetes,Uninsured,Elderly_Medicare,Disabled_Medicare,Prim_Care_Phys_Rate,Dentist_Rate,Community_Health_Center_Ind,HPSA_Ind
count,3141.000000,3141.000000,3141.000000,3141.000000,3.141000e+03,3.141000e+03,3.141000e+03,3141.000000,3141.000000,3141.000000,...,3141.000000,3141.000000,3141.000000,3.141000e+03,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000
mean,30.304680,103.716651,44.696275,38.486151,9.436816e+04,6.320680e+04,1.338754e+05,249.119389,55.554600,584.680357,...,-142.517765,-145.820376,-139.189048,1.264481e+04,10878.690544,1858.450812,57.562464,31.447087,1.550780,1.758994
std,15.134423,107.999484,25.118434,10.290195,3.064317e+05,1.554573e+05,4.025145e+05,1703.041884,113.038973,2345.672274,...,381.651636,380.347217,382.975042,5.460478e+04,31387.569383,4698.552677,44.793770,45.609986,0.497494,0.427762
min,1.000000,1.000000,1.000000,15.000000,6.200000e+01,5.770000e+02,5.228000e+03,-2222.000000,-2222.000000,-2222.000000,...,-1111.100000,-1111.100000,-1111.100000,-2.222000e+03,-2222.000000,-2222.000000,0.000000,-2222.200000,1.000000,1.000000
25%,18.000000,35.000000,23.000000,32.000000,1.121100e+04,6.602000e+03,2.139800e+04,17.000000,9.000000,25.000000,...,5.000000,1.300000,7.300000,1.549000e+03,1607.000000,275.000000,30.500000,18.700000,1.000000,2.000000
50%,29.000000,79.000000,44.000000,37.000000,2.523500e+04,1.384900e+04,2.841100e+04,44.000000,25.000000,80.000000,...,7.000000,3.600000,10.200000,3.426000e+03,3448.000000,690.000000,50.600000,30.000000,2.000000,2.000000
75%,45.000000,133.000000,66.000000,45.000000,6.404000e+04,5.474600e+04,9.419800e+04,109.000000,43.000000,238.000000,...,9.100000,5.200000,13.700000,8.116000e+03,8053.000000,1667.000000,74.700000,43.300000,2.000000,2.000000
max,56.000000,840.000000,88.000000,62.000000,9.935475e+06,1.214361e+06,3.635528e+06,69390.000000,682.000000,20520.000000,...,20.800000,12.300000,30.200000,2.167891e+06,911590.000000,118190.000000,581.200000,377.900000,2.000000,2.000000


In [27]:

tweaker (full[["State_FIPS_Code", "County_FIPS_Code", "CHSI_State_Name"]])

      State_FIPS_Code  State_FIPS_Code  State_FIPS_Code  State_FIPS_Code  \
0                   1                1                1                1   
1                   1                1                1                1   
2                   1                1                1                1   
3                   1                1                1                1   
4                   1                1                1                1   
5                   1                1                1                1   
6                   1                1                1                1   
7                   1                1                1                1   
8                   1                1                1                1   
9                   1                1                1                1   
10                  1                1                1                1   
11                  1                1                1                1   
12          

In [23]:
tweaked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 18 columns):
State_FIPS_Code     3141 non-null int64
State_FIPS_Code     3141 non-null int64
State_FIPS_Code     3141 non-null int64
State_FIPS_Code     3141 non-null int64
State_FIPS_Code     3141 non-null int64
State_FIPS_Code     3141 non-null int64
County_FIPS_Code    3141 non-null int64
County_FIPS_Code    3141 non-null int64
County_FIPS_Code    3141 non-null int64
County_FIPS_Code    3141 non-null int64
County_FIPS_Code    3141 non-null int64
County_FIPS_Code    3141 non-null int64
CHSI_State_Name     3141 non-null object
CHSI_State_Name     3141 non-null object
CHSI_State_Name     3141 non-null object
CHSI_State_Name     3141 non-null object
CHSI_State_Name     3141 non-null object
CHSI_State_Name     3141 non-null object
dtypes: int64(12), object(6)
memory usage: 441.8+ KB
